In [21]:
import boto3
import configparser
import json
import pandas as pd
import os
from time import time

In [2]:
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
REGION                 = config.get('AWS', 'REGION')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

DWH_ENDPOINT           = config.get("CLUSTER", "HOST")


(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)
pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,
4,DWH_DB,dwh
5,DWH_DB_USER,dwhadmin
6,DWH_DB_PASSWORD,
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole


In [3]:
import boto3

ec2 = boto3.resource('ec2',
                       region_name=REGION,
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                       region_name=REGION,
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

iam = boto3.client('iam',aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET,
                     region_name=REGION
                     
                  )

redshift = boto3.client('redshift',
                       region_name=REGION,
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                       )

In [4]:
bucket = s3.Bucket('udacity-dend')

for obj in bucket.objects.filter(Prefix="log-data"):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-01-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-02-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-03-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-04-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-05-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-06-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-07-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-08-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-09-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-10-events.json')
s3.ObjectSummary(b

In [5]:
log_data_files = [filename.key for filename in bucket.objects.filter(Prefix="log-data")]
log_data_files[:10]

['log-data/',
 'log-data/2018/11/2018-11-01-events.json',
 'log-data/2018/11/2018-11-02-events.json',
 'log-data/2018/11/2018-11-03-events.json',
 'log-data/2018/11/2018-11-04-events.json',
 'log-data/2018/11/2018-11-05-events.json',
 'log-data/2018/11/2018-11-06-events.json',
 'log-data/2018/11/2018-11-07-events.json',
 'log-data/2018/11/2018-11-08-events.json',
 'log-data/2018/11/2018-11-09-events.json']

In [6]:
bucket = s3.Bucket('udacity-dend')

for obj in bucket.objects.filter(Prefix="song-data/A/A"):
    print(obj)

s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAK128F9318786.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAV128F421A322.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAABD128F429CF47.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAACN128F9355673.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEA128F935A30D.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAED128E0783FAB.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEM128F93347B9.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEW128F42930C0.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAFD128F92F423A.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAGR128F425B14B.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAHD128F426

In [7]:
song_data_files = [filename.key for filename in bucket.objects.filter(Prefix='song-data/A/A')]
song_data_files[:10]

['song-data/A/A/A/TRAAAAK128F9318786.json',
 'song-data/A/A/A/TRAAAAV128F421A322.json',
 'song-data/A/A/A/TRAAABD128F429CF47.json',
 'song-data/A/A/A/TRAAACN128F9355673.json',
 'song-data/A/A/A/TRAAAEA128F935A30D.json',
 'song-data/A/A/A/TRAAAED128E0783FAB.json',
 'song-data/A/A/A/TRAAAEM128F93347B9.json',
 'song-data/A/A/A/TRAAAEW128F42930C0.json',
 'song-data/A/A/A/TRAAAFD128F92F423A.json',
 'song-data/A/A/A/TRAAAGR128F425B14B.json']

In [8]:
try:
    print('1.1 Creating a new IAM Role')
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement': [{'Action': 'sts:AssumeRole',
               'Effect': 'Allow',
               'Principal': {'Service': 'redshift.amazonaws.com'}}],
             'Version': '2012-10-17'})
    )    
except Exception as e:
    print(e)
    dwhRole = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)

1.1 Creating a new IAM Role
An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name dwhRole already exists.


In [9]:
print('1.2 Attaching Policy')
iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                   PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                   )['ResponseMetadata']['HTTPStatusCode']

1.2 Attaching Policy


200

In [10]:
print('1.3 Get the IAM role ARN')
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']
print(roleArn)

1.3 Get the IAM role ARN
arn:aws:iam::785448629226:role/dwhRole


In [11]:
try:
    response = redshift.create_cluster(
         #HW
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),

        #Identifiers & Credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        
        #Roles (for s3 access)
        IamRoles=[roleArn] 
          )
except Exception as e:
    print(e) 

In [25]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', None)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

NameError: name 'redshift' is not defined

In [8]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

NameError: name 'myClusterProps' is not defined

In [15]:

try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    print(list(vpc.security_groups.all()))
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName= defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

[ec2.SecurityGroup(id='sg-c5bfb6a1')]
ec2.SecurityGroup(id='sg-c5bfb6a1')
An error occurred (InvalidPermission.Duplicate) when calling the AuthorizeSecurityGroupIngress operation: the specified rule "peer: 0.0.0.0/0, TCP, from port: 5439, to port: 5439, ALLOW" already exists


In [3]:
%load_ext sql

In [5]:
pd.DataFrame({"Param":
                  ["DWH_ENDPOINT", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_ENDPOINT,  DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_ENDPOINT,.
1,DWH_CLUSTER_IDENTIFIER,
2,DWH_DB,dwh
3,DWH_DB_USER,dwhadmin
4,DWH_DB_PASSWORD,
5,DWH_PORT,5439
6,DWH_IAM_ROLE_NAME,dwhRole


In [6]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

postgresql://dwhadmin:us-east-2.redshift.amazonaws.com:5439/dwh


'Connected: dwhadmin@dwh'

In [7]:
%%sql
count_user << select count(*) as total_user from dim_user

 * postgresql://dwhadmin:***us-east-2.redshift.amazonaws.com:5439/dwh
1 rows affected.
Returning data to local variable count_user


In [8]:
%%sql
count_song << select count(*) as total_song from dim_song

 * postgresql://dwhadmin:.us-east-2.redshift.amazonaws.com:5439/dwh
1 rows affected.
Returning data to local variable count_song


In [9]:
%%sql
count_artist << select count(*) as total_artist from dim_artist

 * postgresql://dwhadmin:.us-east-2.redshift.amazonaws.com:5439/dwh
1 rows affected.
Returning data to local variable count_artist


In [10]:
%%sql
count_time << select count(*) as total_time from dim_time

 * postgresql://dwhadmin:us-east-2.redshift.amazonaws.com:5439/dwh
1 rows affected.
Returning data to local variable count_time


In [11]:
%%sql
count_songplay << select count(*) as total_songplay from fact_songplay

 * postgresql://dwhadmin:us-east-2.redshift.amazonaws.com:5439/dwh
1 rows affected.
Returning data to local variable count_songplay


In [12]:
print("Total User = " + str(count_user))
print("Total Song = " + str(count_song))
print("Total Artist = " + str(count_artist))
print("Total Time = " + str(count_time))
print("Total Songplay = " + str(count_songplay))

Total User = +------------+
| total_user |
+------------+
|    105     |
+------------+
Total Song = +------------+
| total_song |
+------------+
|   14896    |
+------------+
Total Artist = +--------------+
| total_artist |
+--------------+
|    10025     |
+--------------+
Total Time = +------------+
| total_time |
+------------+
|   40115    |
+------------+
Total Songplay = +----------------+
| total_songplay |
+----------------+
|      333       |
+----------------+


In [15]:
pd.DataFrame({"Param":
                  ["Total User", "Total Song", "Total Artist", "Total Time", "Total Songplay"],
              "Value":
                  [count_user,  count_song, count_artist, count_time, count_songplay]             })

,Param,Value
0,Total User,[(105)]
1,Total Song,[(14896)]
2,Total Artist,[(10025)]
3,Total Time,[(40115)]
4,Total Songplay,[(333)]


In [16]:
%%sql
Stg_Event_Cnt << select count(*) as Stg_Event_Cnt from stg_event


 * postgresql://dwhadmin:us-east-2.redshift.amazonaws.com:5439/dwh
1 rows affected.
Returning data to local variable Stg_Event_Cnt


In [17]:
Stg_Event_Cnt

stg_event_cnt
8056


In [18]:
%%sql
Stg_Song_Cnt << select count(*) as Stg_Song_Cnt from stg_song

 * postgresql://dwhadmin:us-east-2.redshift.amazonaws.com:5439/dwh
1 rows affected.
Returning data to local variable Stg_Song_Cnt


In [19]:
Stg_Song_Cnt

stg_song_cnt
14896


In [24]:
redshift.delete_cluster( ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,  SkipFinalClusterSnapshot=True)
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

NameError: name 'redshift' is not defined